In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
%pip install git+https://github.com/databrickslabs/dbignite.git

  Cloning https://github.com/databrickslabs/dbignite.git to /tmp/pip-req-build-421vdje8
  Running command git clone --filter=blob:none --quiet https://github.com/databrickslabs/dbignite.git /tmp/pip-req-build-421vdje8
  Resolved https://github.com/databrickslabs/dbignite.git to commit 70110d54a662b4c81e0d0974e4bdc1f0f3327452
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dbignite: filename=dbignite-0.2.4-py3-none-any.whl size=8445771 sha256=b1fa87bea433528ad4dc08f259b3a93878969b80851d9c00d96da78984ab6880
  Stored in directory: /tmp/pip-ephem-wheel-cache-f7ifsvr1/wheels/1e/02/ed/34f5ddf9601c002a9502588bfced03057f966c24afb78af73d
Successfully built dbignite
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
# Databricks Secret Scope for the Run-As User
dbutils.widgets.text(
  "databricks_secret_scope"
  ,spark.sql("select current_user()").collect()[0][0].split(sep="@")[0].replace(".", "-")
  ,"User's Personal DB Secrets Scope"
)

# Databricks Secret Key for the GitHub PAT in the User's Scope
dbutils.widgets.text(
  "github_token_secret_key"
  ,"gh_pat"
  ,"GitHub Token DB Secret Key"
)

# Run-As User's GitHub User Name
dbutils.widgets.text(
  "github_username"
  ,spark.sql("select current_user()").collect()[0][0]
  ,"User's Personal GitHub Username"
)

In [0]:

from urllib.parse import quote

github_username = quote(dbutils.widgets.get("github_username"))
github_token = quote(dbutils.secrets.get(scope = dbutils.widgets.get("databricks_secret_scope"), key = dbutils.widgets.get("github_token_secret_key")))

print(f"""
   github_username: {github_username}
   github_token: {github_token}   
""")


   github_username: matthew.giglia%40databricks.com
   github_token: [REDACTED]   



## Connect to Redox

In [0]:
redox_private_key = dbutils.secrets.get(scope = dbutils.widgets.get("databricks_secret_scope"), key = "redox_private_key")
redox_client_id = dbutils.secrets.get(scope = dbutils.widgets.get("databricks_secret_scope"), key = "redox_client_id")
# redox_source_id = dbutils.secrets.get(scope = dbutils.widgets.get("databricks_secret_scope"), key = "redox_source_id")
redox_source_id = "91b0ab2f-7b86-441d-9cbf-9b9a6d648d59"
redox_public_kid = dbutils.secrets.get(scope = dbutils.widgets.get("databricks_secret_scope"), key = "redox_kid")

print(f""" 
      redox_private_key: {redox_private_key}
      redox_client_id: {redox_client_id}
      redox_source_id: {redox_source_id}
      redox_public_kid: {redox_public_kid}
""")

 
      redox_private_key: [REDACTED]
      redox_client_id: [REDACTED]
      redox_source_id: 91b0ab2f-7b86-441d-9cbf-9b9a6d648d59
      redox_public_kid: [REDACTED]



In [0]:
import json

redox_auth_json = f"""
{{
  "kty": "RSA",
  "kid": "{redox_public_kid}",
  "alg": "RS384",
  "use": "sig"
}}
"""

json.loads(redox_auth_json)

{'kty': 'RSA',
 'kid': '[REDACTED]',
 'alg': 'RS384',
 'use': 'sig'}

In [0]:
from redoxwrite.auth import * 
from redoxwrite.endpoint import *

auth = RedoxApiAuth(
  redox_client_id
  ,redox_private_key
  ,redox_auth_json
  ,redox_source_id
)
print("Is connection successful? " + str(auth.can_connect()))

Is connection successful? True


In [0]:
#All Redox FHIR request URLs start with this base: https://api.redoxengine.com/fhir/R4/[organization-name]/[environment-type]/
redox_base_url = 'https://api.redoxengine.com/fhir/R4/redox-fhir-sandbox/Development/'

rapi = RedoxApiRequest(auth, base_url = redox_base_url)

In [0]:
result = rapi.make_request("post", resource="Patient", action="_search", data = "{'identifier' : 'urn:redox:redox-fhir-sandbox:MR|kyHGADnvX3xbkU4V9ayaqh'}")

In [0]:
if result['response']['response_status_code'] != 200:
  print("Failed to update the patient information")
print(json.dumps(json.loads(result['response']['response_text']), indent=2))

{
  "entry": [
    {
      "fullUrl": "https://fhir.redoxengine.com/fhir-sandbox/Patient/a6d89078-6ca4-4f50-8c73-70d60dd01957",
      "resource": {
        "address": [
          {
            "city": "",
            "country": "",
            "line": [
              ""
            ],
            "postalCode": "",
            "state": ""
          }
        ],
        "birthDate": "1990-05-08",
        "id": "a6d89078-6ca4-4f50-8c73-70d60dd01957",
        "identifier": [
          {
            "system": "metadata_entry_id",
            "value": "f32940e5-c4a5-4587-b43d-8e2ca5f65923"
          }
        ],
        "meta": {
          "lastUpdated": "2025-01-20T18:20:23.643294+00:00",
          "versionId": "MTczNzM5NzIyMzY0MzI5NDAwMA"
        },
        "name": [
          {
            "family": "medina",
            "given": [
              "ariel",
              ""
            ]
          }
        ],
        "resourceType": "Patient",
        "telecom": [
          {
            "s

In [0]:
# approved_service = """
# {
#   "id": "RedoxClaimResponseBundleComprehensiveExample",
#   "type": "collection",
#   "entry": [
#     {
#       "resource": {
#         "id": "RedoxPAClaimResponseExample",
#         "use": "preauthorization",
#         "item": [
#           {
#             "extension": [
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-administrationReferenceNumber",
#                 "valueString": "REF456"
#               },
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemPreAuthIssueDate",
#                 "valueDate": "2021-08-23"
#               },
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemPreAuthPeriod",
#                 "valuePeriod": {
#                   "end": "2021-09-23",
#                   "start": "2021-08-23"
#                 }
#               },
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemTraceNumber",
#                 "valueIdentifier": {
#                   "value": "23462346"
#                 }
#               },
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemAuthorizedDetail",
#                 "extension": [
#                   {
#                     "url": "productOrServiceCode",
#                     "valueCodeableConcept": {
#                       "text": "Right heart catheterization",
#                       "coding": [
#                         {
#                           "code": "93451",
#                           "system": "http://www.ama-assn.org/go/cpt"
#                         }
#                       ]
#                     }
#                   },
#                   {
#                     "url": "quantity",
#                     "valueQuantity": {
#                       "value": 1
#                     }
#                   },
#                   {
#                     "url": "unitPrice",
#                     "valueMoney": {
#                       "value": 4966,
#                       "currency": "USD"
#                     }
#                   }
#                 ]
#               }
#             ],
#             "adjudication": [
#               {
#                 "category": {
#                   "coding": [
#                     {
#                       "code": "submitted",
#                       "system": "http://terminology.hl7.org/CodeSystem/adjudication"
#                     }
#                   ]
#                 },
#                 "extension": [
#                   {
#                     "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-reviewAction",
#                     "extension": [
#                       {
#                         "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-reviewActionCode",
#                         "valueCodeableConcept": {
#                           "coding": [
#                             {
#                               "code": "A1",
#                               "system": "https://codesystem.x12.org/005010/306",
#                               "display": "Certified in total"
#                             }
#                           ]
#                         }
#                       },
#                       {
#                         "url": "number",
#                         "valueString": "AUTH1001"
#                       }
#                     ]
#                   }
#                 ]
#               }
#             ],
#             "itemSequence": 1
#           }
#         ],
#         "type": {
#           "coding": [
#             {
#               "code": "professional",
#               "system": "http://terminology.hl7.org/CodeSystem/claim-type"
#             }
#           ]
#         },
#         "status": "active",
#         "created": "2021-08-23T16:35:54.648Z",
#         "insurer": {
#           "reference": "Organization/RedoxPAInsurerOrganizationExample"
#         },
#         "outcome": "complete",
#         "patient": {
#           "reference": "Patient/RedoxPABeneficiaryExample"
#         },
#         "request": {
#           "reference": "Claim/RedoxPAServiceClaimExample",
#           "identifier": {
#             "type": {
#               "coding": [
#                 {
#                   "code": "ClaimID"
#                 }
#               ]
#             },
#             "value": "urn:uuid:97b8d6d0-ac3d-411a-b32a-e1d237aceb6a"
#           }
#         },
#         "requestor": {
#           "reference": "Organization/RedoxPARequestorOrganizationExample"
#         },
#         "processNote": [
#           {
#             "text": "Covered under extended benefits"
#           }
#         ],
#         "resourceType": "ClaimResponse"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPABeneficiaryExample",
#         "name": [
#           {
#             "given": [
#               "Timothy",
#               "Paul"
#             ],
#             "family": "Bixby"
#           }
#         ],
#         "gender": "male",
#         "birthDate": "2008-01-06",
#         "identifier": [
#           {
#             "value": "M23462346",
#             "system": "urn:redox:MRN"
#           }
#         ],
#         "resourceType": "Patient"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPACoverageWithRelatedSubscriberExample",
#         "class": [
#           {
#             "type": {
#               "coding": [
#                 {
#                   "code": "group",
#                   "system": "http://terminology.hl7.org/CodeSystem/coverage-class"
#                 }
#               ]
#             },
#             "value": "GRP2245"
#           }
#         ],
#         "payor": [
#           {
#             "reference": "Organization/RedoxPAInsurerOrganizationExample"
#           }
#         ],
#         "status": "active",
#         "subscriber": {
#           "reference": "RelatedPerson/RedoxPASubscriberExample"
#         },
#         "beneficiary": {
#           "reference": "Patient/RedoxPABeneficiaryExample"
#         },
#         "resourceType": "Coverage",
#         "subscriberId": "M2346623"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPARequestorOrganizationExample",
#         "name": "Good Health Clinics",
#         "active": true,
#         "address": [
#           {
#             "city": "Madison",
#             "line": [
#               "123 Main St."
#             ],
#             "state": "WI",
#             "country": "USA",
#             "district": "Dane",
#             "postalCode": "53703"
#           }
#         ],
#         "resourceType": "Organization"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPAInsurerOrganizationExample",
#         "name": "Aetna",
#         "active": true,
#         "address": [
#           {
#             "city": "Lexington",
#             "line": [
#               "PO Box 14080"
#             ],
#             "state": "KY",
#             "country": "USA",
#             "district": "Fayette",
#             "postalCode": "40512-4079"
#           }
#         ],
#         "resourceType": "Organization"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPAPractitionerRoleExamplePat",
#         "location": [
#           {
#             "display": "1st Street Family Clinic"
#           }
#         ],
#         "specialty": [
#           {
#             "coding": [
#               {
#                 "code": "394579002",
#                 "system": "http://snomed.info/sct",
#                 "display": "Cardiology"
#               }
#             ]
#           }
#         ],
#         "organization": {
#           "reference": "Organization/RedoxPARequestorOrganizationExample"
#         },
#         "practitioner": {
#           "reference": "Practitioner/RedoxPAPractitionerExamplePat"
#         },
#         "resourceType": "PractitionerRole"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPAPractitionerExamplePat",
#         "name": [
#           {
#             "given": [
#               "Pat"
#             ],
#             "family": "Granite"
#           }
#         ],
#         "address": [
#           {
#             "city": "Madison",
#             "line": [
#               "123 Main St."
#             ],
#             "state": "WI",
#             "country": "USA",
#             "district": "Dane",
#             "postalCode": "53703"
#           }
#         ],
#         "telecom": [
#           {
#             "value": "+16085551234"
#           }
#         ],
#         "identifier": [
#           {
#             "value": "4356789876",
#             "system": "http://hl7.org/fhir/sid/us-npi"
#           }
#         ],
#         "resourceType": "Practitioner"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPASubscriberExample",
#         "name": [
#           {
#             "given": [
#               "Barbara"
#             ],
#             "family": "Bixby"
#           }
#         ],
#         "patient": {
#           "reference": "RedoxPABeneficiary"
#         },
#         "relationship": [
#           {
#             "text": "Mother"
#           }
#         ],
#         "resourceType": "RelatedPerson"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPAServiceClaimExample",
#         "use": "preauthorization",
#         "item": [
#           {
#             "sequence": 1,
#             "extension": [
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-requestedService",
#                 "valueReference": {
#                   "reference": "ServiceRequest/RedoxPAServiceRequestExample"
#                 }
#               },
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemTraceNumber",
#                 "valueIdentifier": {
#                   "value": "23462346"
#                 }
#               }
#             ],
#             "productOrService": {
#               "text": "Coronary angiography with right heart catheterization",
#               "coding": [
#                 {
#                   "code": "93456",
#                   "system": "http://www.ama-assn.org/go/cpt"
#                 }
#               ]
#             }
#           }
#         ],
#         "type": {
#           "coding": [
#             {
#               "code": "professional",
#               "system": "http://terminology.hl7.org/CodeSystem/claim-type"
#             }
#           ]
#         },
#         "status": "active",
#         "created": "2021-09-02T18:44:13.307Z",
#         "enterer": {
#           "reference": "PractitionerRole/RedoxPAPractitionerRoleExamplePat"
#         },
#         "insurer": {
#           "reference": "Organization/RedoxPAInsurerOrganizationExample"
#         },
#         "patient": {
#           "reference": "Patient/RedoxPABeneficiaryExample"
#         },
#         "priority": {
#           "coding": [
#             {
#               "code": "normal",
#               "system": "http://terminology.hl7.org/CodeSystem/processpriority"
#             }
#           ]
#         },
#         "provider": {
#           "reference": "Organization/RedoxPARequestorOrganizationExample"
#         },
#         "diagnosis": [
#           {
#             "sequence": 1,
#             "diagnosisCodeableConcept": {
#               "coding": [
#                 {
#                   "code": "I42.9",
#                   "system": "http://hl7.org/fhir/sid/icd-10-cm",
#                   "display": "Cardiomyopathy, unspecified"
#                 }
#               ]
#             }
#           }
#         ],
#         "insurance": [
#           {
#             "focal": true,
#             "coverage": {
#               "reference": "Coverage/RedoxPACoverageExample"
#             },
#             "sequence": 1
#           }
#         ],
#         "identifier": [
#           {
#             "type": {
#               "coding": [
#                 {
#                   "code": "ClaimID"
#                 }
#               ]
#             },
#             "value": "urn:uuid:97b8d6d0-ac3d-411a-b32a-e1d237aceb6a"
#           }
#         ],
#         "resourceType": "Claim",
#         "supportingInfo": [
#           {
#             "category": {
#               "coding": [
#                 {
#                   "code": "additionalInformation",
#                   "system": "http://hl7.org/fhir/us/davinci-pas/CodeSystem/PASSupportingInfoType",
#                   "display": "Send additional paperwork or supporting information is sent for the request."
#                 }
#               ]
#             },
#             "sequence": 1,
#             "valueReference": {
#               "reference": "DocumentReference/RedoxPADocumentReferenceExample"
#             }
#           },
#           {
#             "category": {
#               "coding": [
#                 {
#                   "code": "freeFormMessage",
#                   "system": "http://hl7.org/fhir/us/davinci-pas/CodeSystem/PASSupportingInfoType",
#                   "display": "Written Confirmation."
#                 }
#               ]
#             },
#             "sequence": 2,
#             "valueString": "1"
#           }
#         ]
#       }
#     }
#   ],
#   "timestamp": "2021-10-21T19:09:02.452Z",
#   "identifier": {
#     "value": "urn:uuid:59f129cf-2eca-4965-88f2-452558af1934",
#     "system": "urn:ietf:rfc:3986"
#   },
#   "resourceType": "Bundle"
# }
# """

In [0]:
# result = rapi.make_request(http_method="post", resource="ClaimResponse", action="$respond", data=approved_service)

In [0]:
# result

In [0]:
# if result['response']['response_status_code'] != 200:
#   print("Failed to update the patient information")
# print(json.dumps(json.loads(result['response']['response_text']), indent=2))

In [0]:
enrollment_sdf = spark.table("mgiglia.hv_claims.enrollment")
display(enrollment_sdf)

patient_id,patient_gender,patient_year_of_birth,patient_zip3,patient_state,date_start,date_end,benefit_type,pay_type
0025d1cd1e33cbb0a4549a6ee16211a0,M,2010,711,LA,2019-01-01,2024-02-29,MEDICAL,MEDICAID
0025d1cd1e33cbb0a4549a6ee16211a0,M,2010,711,LA,2019-01-01,2024-02-29,PHARMACY,MEDICAID
003bb14eb5cd1e5894b0a297ba5c8a54,M,1934,641,MO,2017-01-01,2023-11-30,MEDICAL,MEDICARE
003bb14eb5cd1e5894b0a297ba5c8a54,M,1934,641,MO,2017-01-01,2023-11-30,PHARMACY,MEDICARE
0043e20418ced8e6614abf486174ad8b,F,1984,833,ID,2017-01-01,2024-02-29,MEDICAL,COMMERCIAL
0043e20418ced8e6614abf486174ad8b,F,1984,833,ID,2017-01-01,2024-02-29,PHARMACY,COMMERCIAL
00678cc272e5d05ae562f6229444689a,M,1934,452,OH,2017-01-01,2023-12-31,MEDICAL,MEDICARE
00678cc272e5d05ae562f6229444689a,M,1934,452,OH,2017-01-01,2023-12-31,PHARMACY,MEDICARE
00932abdd327a37000be6aed8c6ec984,M,1982,436,OH,2017-01-01,2024-02-29,MEDICAL,COMMERCIAL
00932abdd327a37000be6aed8c6ec984,M,1982,436,OH,2017-01-01,2024-02-29,PHARMACY,COMMERCIAL


In [0]:
from dbignite.writer.bundler import *
from dbignite.writer.fhir_encoder import *

In [0]:
help(Mapping)

Help on class Mapping in module dbignite.writer.fhir_encoder:

class Mapping(builtins.object)
 |  Mapping(src, tgt, hardcoded=False)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, src, tgt, hardcoded=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  fhir_resource(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables
 |  
 |  __weakref__
 |      list of weak references to the object



In [0]:
maps = [
  Mapping('patient_id', 'Patient.id')
  ,Mapping('patient_gender', 'Patient.gender')
  ,Mapping('patient_year_of_birth', 'Patient.birthDate')
  ,Mapping('patient_zip3', 'Patient.address.postalCode')
  ,Mapping('patient_state', 'Patient.address.state')
  ,Mapping('date_start', 'Coverage.period.start')
  ,Mapping('date_end', 'Coverage.period.end')
  ,Mapping()
  ,Mapping('<url of a hardcoded system reference>', 'Patient.identifier.system', True)
	,Mapping('PATIENT_ID', 'Patient.id')]